# Prompt Engineering

*This notebook is modeled after my [NLP Prompt Engineer R Journal](https://mpfoley73.github.io/nlp/prompt-engineering.html).*

An API token is required for OpenAI’s ChatGPT API service. I created one at https://platform.openai.com/api-keys and saved it to .env.

In [5]:
import os
import openai

openai.api_key=os.environ.get("OPENAI_API_KEY")

It is possible to elicit responses from OpenAI models in a particular format. Large models like GPT-3.5 will adapt their responses to the format you specify. Let’s try a few examples. First, set up a function to make the call.

In [11]:
def get_openai_response(context, user_message):

    message_list = context.copy()
    message_list.append(
        {'role':'user', 'content':"question: " + user_message}
    )

    response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=message_list,
            temperature=1,
        )

    return (response.choices[0].message.content)

A standard prompt with system message like “You are an expert in Major League Baseball.” followed by user message “Who won the 2016 World Series?” might return a free-form text response like this:

In [12]:
context = [
    {'role':'system', 'content':'You are an expert in Major League Baseball.'},
]
print(get_openai_response(context, 'Who won the 2016 World Series?'))

The Chicago Cubs won the 2016 World Series. It was their first championship in 108 years, and they defeated the Cleveland Indians in a thrilling seven-game series.


This is called _zero-shot_ because there is no demonstration of what the output should look like. A _one-shot_ prompt might look like this:

In [14]:
context = [
    {'role':'system', 'content':
     """You are an expert in Major League Baseball.

    Who won the 1995 World Series?
    Winning Team: Atlanta Braves.
    Losing Team: Cleveland Indians."""}
]
print(get_openai_response(context, 'Who won the 2016 World Series?'))

The 2016 World Series was won by the Chicago Cubs. They defeated the Cleveland Indians to claim their first championship in 108 years.


That didn’t work. How about a two-shot?

In [15]:
context = [
    {'role':'system', 'content':
     """You are an expert in Major League Baseball.

    Who won the 1995 World Series?
    Winning Team: Atlanta Braves.
    Losing Team: Cleveland Indians.
    Who won the 1997 World Series?
    Winning Team: Florida Marlins.
    Losing Team: Cleveland Indians."""}
]
print(get_openai_response(context, 'Who won the 2016 World Series?'))

The Chicago Cubs won the 2016 World Series.


Thought that would do it… There is another way. The model learns best by conversation. Use the assistant role to help it along. This is the proper way to build a prompt.

In [16]:
context = [
    {'role':'system', 'content':'You are an expert in Major League Baseball.\n\n'},
    {'role':'user', 'content':'Who won the 1995 World Series?'},
    {'role':'assistant', 'content':"""Winning Team: Atlanta Braves. \nLosing Team: Cleveland Indians. """},
    {'role':'user', 'content':'Who won the 1997 World Series?'},
    {'role':'assistant', 'content':"""Winning Team: Florida Marlins. \nLosing Team: Cleveland Indians. """},
]
print(get_openai_response(context, 'Who won the 2016 World Series?'))

Winning Team: Chicago Cubs. 
Losing Team: Cleveland Indians.


You would operationalize that by putting the system and user-assistant pairs in the function definition. Still one more option is to supply explicit instructions. Below, I combine explicit instructions with two-shot.

In [17]:
context = [
    {'role':'system', 'content':"""You are an expert in Major League Baseball.
     Answer the question with the winning team and losing team. Use the 
     following format:
     Winning Team: 
     Losing Team: """},
    {'role':'user', 'content':'Who won the 1995 World Series?'},
    {'role':'assistant', 'content':"""Winning Team: Atlanta Braves. \nLosing Team: Cleveland Indians. """},
    {'role':'user', 'content':'Who won the 1997 World Series?'},
    {'role':'assistant', 'content':"""Winning Team: Florida Marlins. \nLosing Team: Cleveland Indians. """},
]
print(get_openai_response(context, 'Who won the 2016 World Series?'))

Winning Team: Chicago Cubs. 
Losing Team: Cleveland Indians.
